## Competative Social Media Analysis

**Explore Data Science Academy** wants to develop a marketing startegy to ensure that all social media content posted reaches the inteded target market or audiance and that our social media market are growing in engagement and also analysing our growing social media engagement as compared to the competitors.

#### Import Standard libraries

In [124]:
import re
import csv
import pandas as pd

import selenium
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as bs

# for visualization
import seaborn as sns
import matplotlib.pyplot as plt

# for extraction
from geograpy import extraction
import nltk
nltk.download('maxent_ne_chunker')

# for organization
from collections import Counter

import time
from datetime import datetime


[nltk_data] Error loading maxent_ne_chunker: <urlopen error [Errno -2]
[nltk_data]     Name or service not known>


#### Configurations

In [128]:
import warnings
warnings.filterwarnings('ignore')

#Disable chrome Notifications
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")

#Install latest chrome drivers
#driver = webdriver.chrome(service=Service(ChromeDriverManager().install()))

#Thanks stackoverflow @pythonjar
#chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

#specify the path to chromedriver.exe (download and save on your computer)
browser = webdriver.Chrome('/home/nano-ai/chromedriver' , chrome_options=chrome_options)

### Automatic Login to Social Media

In [129]:
#Get the Company's  URL from User
page = input("Enter the Company Linkedin URL: ")
company_name = page[33:-1]

#See if existing user credential file exists or create one 
try:
    f= open("linkedin_credentials.txt","r")
    contents = f.read()
    username = contents.replace("=",",").split(",")[1]
    password = contents.replace("=",",").split(",")[3]
except:
    f= open("linkedin_credentials.txt","w+")
    username = input('Enter your linkedin username: ')
    password = input('Enter your linkedin password: ')
    f.write("username={}, password={}".format(username,password))
    f.close()


Enter the Company Linkedin URL: https://www.linkedin.com/school/udacity/posts/?feedView=all


#### Navigate to facebook page/group

In [130]:
#accessing Chromedriver
# browser = webdriver.Chrome('chromedriver')
browser.maximize_window()
time.sleep(2)

#Open login page
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Enter login info:
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)
elementID.submit()

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=100.0.4896.127)
Stacktrace:
#0 0x55940afc37d3 <unknown>
#1 0x55940ad1f688 <unknown>
#2 0x55940ad19640 <unknown>
#3 0x55940ad0ad26 <unknown>
#4 0x55940ad0bc0e <unknown>
#5 0x55940ad0aff2 <unknown>
#6 0x55940ad0a4a5 <unknown>
#7 0x55940ad09190 <unknown>
#8 0x55940ad095b2 <unknown>
#9 0x55940ad2104a <unknown>
#10 0x55940ad8726f <unknown>
#11 0x55940ad736b2 <unknown>
#12 0x55940ad86a0c <unknown>
#13 0x55940ad735a3 <unknown>
#14 0x55940ad48ddc <unknown>
#15 0x55940ad49de5 <unknown>
#16 0x55940aff449d <unknown>
#17 0x55940b00d60c <unknown>
#18 0x55940aff6205 <unknown>
#19 0x55940b00dee5 <unknown>
#20 0x55940afea070 <unknown>
#21 0x55940b029488 <unknown>
#22 0x55940b02960c <unknown>
#23 0x55940b042c6d <unknown>
#24 0x7fae9324d609 <unknown>


In [ ]:
#Go to company post webpage
browser.get(page)


#Simulate scrolling to capture all posts
SCROLL_PAUSE_TIME = 1.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

### Check page source code

In [ ]:
#Check out page source code
company_page = browser.page_source  


#Use Beautiful Soup to get access tags
linkedin_soup = bs(company_page.encode("utf-8"), "html")
linkedin_soup.prettify()

#Find the post blocks
containers = linkedin_soup.findAll("div",{"class":"occludable-update ember-view"})

### Data Scraping

In [ ]:
<div class="org-top-card-summary-info-list__info-item">
        78,339+ alumni
      </div>

In [ ]:
# Lists that we will iterate to
post_dates = []
post_texts = []
post_likes = []
post_comments = []
video_views = []
media_links = []
media_type = []
company = []
followers = []
alumni = []
place = []


#Looping through the posts and appending them to the lists
for container in containers:
    
    #Try function to make sure its a post and not a promotion
    try:
        posted_date = container.find("span",{"class":"visually-hidden"})
        text_box = container.find("div",{"class":"feed-shared-text relative feed-shared-update-v2__commentary"})
        text = text_box.find("span",{"dir":"ltr"})
        new_likes = container.findAll("span", {"class":"social-details-social-counts__reactions-count"})
        new_comments = container.findAll("li", {"class": "social-details-social-counts__item social-details-social-counts__comments social-details-social-counts__item--with-social-proof"})
        company_name = container.findAll("span" , {"class":"feed-shared-actor__name t-14 t-bold hoverable-link-text t-black"})
        num_followers = container.findAll("span" , {"feed-shared-actor__description t-12 t-normal t-black--light"})
        company_place = container.findAll("div" , {"class":"org-top-card-summary-info-list__info-item"})
        num_alumni = container.findAll("div" , {"class":"org-top-card-summary-info-list__info-item"})

        
        #Appending date and text to lists
        post_dates.append(posted_date.text.strip())
        post_texts.append(text_box.text.strip())


        #Determining media type and collecting relevant info for each type
        try:
            video_box = container.findAll("div",{"class": "feed-shared-update-v2__content feed-shared-linkedin-video ember-view"})
            video_link = video_box[0].find("video", {"class":"vjs-tech"})
            media_links.append(video_link['src'])
            media_type.append("Video")
        except:
            try:
                image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image feed-shared-image__image--constrained lazy-image ember-view"})
                media_links.append(image_link['src'])
                media_type.append("Image")
            except:
                try:
                    image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                    image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image lazy-image ember-view"})
                    media_links.append(image_link['src'])
                    media_type.append("Image")
                except:
                    try:
                        article_box = container.findAll("div",{"class": "feed-shared-article__description-container"})
                        article_link = article_box[0].find('a', href=True)
                        media_links.append(article_link['href'])
                        media_type.append("Article")
                    except:
                        try:
                            video_box = container.findAll("div",{"class": "feed-shared-external-video__meta"})          
                            video_link = video_box[0].find('a', href=True)
                            media_links.append(video_link['href'])
                            media_type.append("Youtube Video")   
                        except:
                            try:
                                poll_box = container.findAll("div",{"class": "feed-shared-update-v2__content overflow-hidden feed-shared-poll ember-view"})
                                media_links.append("None")
                                media_type.append("Other: Poll, Shared Post, etc")
                            except:
                                media_links.append("None")
                                media_type.append("Unknown")



        #Getting Video Views. (The folling three lines prevents class name overlap)
        view_container2 = set(container.findAll("li", {'class':["social-details-social-counts__item"]}))
        view_container1 = set(container.findAll("li", {'class':["social-details-social-counts__reactions","social-details-social-counts__comments social-details-social-counts__item"]}))
        result = view_container2 - view_container1

        view_container = []
        for i in result:
            view_container += i

        try:
            video_views.append(view_container[1].text.strip())

        except:
            video_views.append('N/A')

        
        #Appending likes and comments if they exist
        try:
            post_likes.append(new_likes[0].text.strip())
        except:
            post_likes.append(0)
            pass

        try:
            post_comments.append(new_comments[0].text.strip())
        except:                                                           
            post_comments.append(0)
            
            pass
        try:
            company.append(company_name[0].text.strip())
            
        except:
            company.append(0)
            
        try:
            followers.append(num_followers[0].text.strip())
            place.append(company_place[0].text.strip())
            alumni.append(new_alumni[0].text.strip())

        except:
            followers.append(0)
            place.append(0)
            alumni.append(0)
    
    except:
        pass


In [ ]:
#Stripping non-numeric values
comment_count = []
for i in post_comments:
    s = str(i).replace('Comment','').replace('s','').replace(' ','')
    comment_count += [s]

In [ ]:
data = {
    "Company": company,
    "Place":place,
    "Date Posted": post_dates,
    "Media Type": media_type,
    "Post Text": post_texts,
    "Post Likes": post_likes,
    "Post Comments": comment_count,
    "Video Views": video_views,
    "Media Links": media_links,
    "Alumni":alumni,
    "Followers":followers
}

df = pd.DataFrame(data)

In [ ]:
df.tail(50)

In [ ]:
data['Company'][0]

### Data Preprocessing

#### Convert Dataframe to CSV File

In [120]:
#Exporting as csv file to program folder
df.to_csv("{}_posts.csv".format(df['Company'][0]), encoding='utf-8', index=False)

#### Save csv file to Coud Storage (Aws Elastic File Storage)

### Exploratory Data Analysis (EDA)